# Map Madaraka Express GTFS Data

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import folium
from folium.features import CustomIcon

# Load GTFS data
agency = pd.read_csv('Madaraka_Express_GTFS/agency.txt')
calendar = pd.read_csv('Madaraka_Express_GTFS/calendar.txt')
routes = pd.read_csv('Madaraka_Express_GTFS/routes.txt')
shapes = pd.read_csv('Madaraka_Express_GTFS/shapes.txt')
stop_times = pd.read_csv('Madaraka_Express_GTFS/stop_times.txt')
stops = pd.read_csv('Madaraka_Express_GTFS/stops.txt')
trips = pd.read_csv('Madaraka_Express_GTFS/trips.txt')

# Create a dictionary to store the shape points
shape_dict = {}

for index, row in shapes.iterrows():
    shape_id = row['shape_id']
    lat, lon = row['shape_pt_lat'], row['shape_pt_lon']
    if shape_id not in shape_dict:
        shape_dict[shape_id] = []
    shape_dict[shape_id].append((lon, lat))

# Create a list of LineStrings
shapes_list = []
for shape_id, points in shape_dict.items():
    shapes_list.append({'shape_id': shape_id, 'geometry': LineString(points)})

# Create a Geopandas DataFrame
gdf_shapes = gpd.GeoDataFrame(shapes_list)

# Create a base map
m = folium.Map(location=[-1.286389, 36.817223], zoom_start=7)  # Centered around Nairobi, Kenya

# Add shapes to the map
for _, row in gdf_shapes.iterrows():
    folium.PolyLine(locations=[(point[1], point[0]) for point in row['geometry'].coords], color='blue', weight=2.5).add_to(m)

# Add stops to the map with custom icons and popups
for _, row in stops.iterrows():
    popup_html = f'<div style="width: 200px;">{row["stop_name"]}</div>'
    popup = folium.Popup(popup_html, max_width=250)
    icon = folium.Icon(icon='train', prefix='fa', color='red')  # Font Awesome train icon
    folium.Marker(
        location=[row['stop_lat'], row['stop_lon']],
        popup=popup,
        icon=icon
    ).add_to(m)

# Save the map to an HTML file
m.save('madaraka_express_map.html')

In [3]:
import pandas as pd

# Function to normalize time
def normalize_time(time_str):
    """Convert time string to a timedelta object handling over-24-hour times."""
    hours, minutes, seconds = map(int, time_str.split(':'))
    return pd.Timedelta(hours=hours, minutes=minutes, seconds=seconds)

# Load GTFS data
agency = pd.read_csv('Madaraka_Express_GTFS/agency.txt')
calendar = pd.read_csv('Madaraka_Express_GTFS/calendar.txt')
fare_attributes = pd.read_csv('Madaraka_Express_GTFS/fare_attributes.txt')
fare_rules = pd.read_csv('Madaraka_Express_GTFS/fare_rules.txt')
feed_info = pd.read_csv('Madaraka_Express_GTFS/feed_info.txt')
routes = pd.read_csv('Madaraka_Express_GTFS/routes.txt')
shapes = pd.read_csv('Madaraka_Express_GTFS/shapes.txt')
stop_times = pd.read_csv('Madaraka_Express_GTFS/stop_times.txt')
stops = pd.read_csv('Madaraka_Express_GTFS/stops.txt')
trips = pd.read_csv('Madaraka_Express_GTFS/trips.txt')

# Normalize arrival and departure times
stop_times['arrival_time'] = stop_times['arrival_time'].apply(normalize_time)
stop_times['departure_time'] = stop_times['departure_time'].apply(normalize_time)

# Merge necessary data
trips_with_shapes = pd.merge(trips, shapes, on='shape_id')
trips_with_shapes_times = pd.merge(trips_with_shapes, stop_times, on='trip_id')
trips_with_shapes_times_stops = pd.merge(trips_with_shapes_times, stops, on='stop_id')

# Sort by departure time
trips_with_shapes_times_stops.sort_values(by='departure_time', inplace=True)

import folium
from datetime import timedelta

# Function to create a map for a specific time
def create_map(time, df):
    # Filter data for the current time
    current_data = df[(df['departure_time'] <= time) & (df['arrival_time'] >= time)]
    
    # Create a map centered on Kenya
    m = folium.Map(location=[-1.2921, 36.8219], zoom_start=6)
    
    # Add trips to the map
    for _, row in current_data.iterrows():
        folium.Marker(
            location=[row['stop_lat'], row['stop_lon']],
            popup=f"Trip {row['trip_id']}, Stop {row['stop_id']}",
        ).add_to(m)
    
    return m

# Create maps for each time step
start_time = normalize_time('00:00:00')
end_time = normalize_time('23:59:59')
time_steps = [start_time + timedelta(hours=i) for i in range(24)]
maps = [create_map(time, trips_with_shapes_times_stops) for time in time_steps]

from selenium import webdriver
from PIL import Image
import imageio
import os

# Function to save a map as an image
def save_map_as_image(map_obj, filename):
    map_obj.save('map.html')
    
    # Configure webdriver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    
    driver.get('file:///' + os.path.realpath('map.html'))
    driver.save_screenshot(filename)
    driver.quit()

# Save maps as images
image_files = []
for i, m in enumerate(maps):
    image_file = f'map_{i}.png'
    save_map_as_image(m, image_file)
    image_files.append(image_file)

# Convert images to GIF
images = [Image.open(image_file) for image_file in image_files]
imageio.mimsave('trips_animation.gif', images, duration=1)

# Clean up
for image_file in image_files:
    os.remove(image_file)
os.remove('map.html')